# MCNP project simulations

Lessons learned from MCNP4
* Plotting window doesn't show boundary errors necessarily intuitively 
* KCODE may require specifying upper bound on alloc space (10x n hist here)
* VOL card ignores cell numbers, and instead uses definition order

In [ ]:
from bokeh import plotting as plt
from bokeh.io import output_notebook
output_notebook()

In [ ]:
import sys
sys.path.append("py_cfg_parsing")

In [ ]:
import pathlib
mcnp_dir = pathlib.Path.cwd() / "mcnp"

# Load secrets

In [ ]:
from plumbum import cmd

lookup_secret = lambda field: cmd.secret_tool("lookup", "app", "phymat", "field", field).strip()
user = lookup_secret("user")
domain = lookup_secret("domain")
port = lookup_secret("port")
password = lookup_secret("password")

# Start SSH file system

In [ ]:
import pexpect
p = pexpect.spawn('bash')
p.sendline(f'sshfs {user}@{domain}:mcnp {mcnp_dir} -p {port}')
p.expect("password:")
p.sendline(password)
p.sendline('exit')
p.expect_exact(pexpect.EOF)

# Define SSH session & commands

In [ ]:
from plumbum.machines.paramiko_machine import ParamikoMachine
    
remote = ParamikoMachine(domain, user, port, password, keep_alive=30)
remote._cwd = remote.cwd.chdir('mcnp')
        
mcnp = remote['./mcnp']

def clean():
    z =  [("-I", n) for n in ("mcnp", "examples", "utils", "pt1.tex", "*.ip")] 
    filenames = remote['ls'](*chain(*z)).splitlines()
    if filenames:
        remote['rm'](*filenames)

# Output highlighting

In [ ]:
from colorama import Fore
from itertools import chain
import re

replacers = {r'warning\.': Fore.YELLOW, r'fatal error\.': Fore.RED, 
             r'\d+ particles got lost.': Fore.RED}

def _highlight(string, replacers):
    pattern = '|'.join(f'({p})' for p in replacers)
    def replacer(m):
        code = next(v for i, v in enumerate(replacers.values()) if m.groups()[i] is not None)
        return f"{code}{m.group(0)}{Fore.RESET}"
    return re.sub(pattern, replacer, string)
    
def prettify(text):
    print(_highlight(text, replacers))

# Part 1

In [ ]:
%%writefile mcnp/1.ip
MESSAGE:

Practical Monte Carlo part 1. Angus Hollands
C
C Cells
1 1 -7.92 1 -2 3 -4 5 -6 (-7:8:-9:10:-11) $ Walls
2 0 (-1:2:-3:4:-5:6) $ Void
3 2 -1.0 7 -8 9 -10 11 -12 $ Water
4 0 7 -8 9 -10 12 -6 $ Air gap (void)

C Surfaces
C Define outer walls
1 PX -5.20
2 PX 5.20
3 PY -10.20
4 PY 10.20
5 PZ 0.0
6 PZ 20.0
C Define XY wall surfaces
7 PX -5.00
8 PX 5.00
9 PY -10.0
10 PY 10.0
C Define Z wall / water surfaces
11 PZ 0.20
12 PZ 19.0

C Cell importance MAP
IMP:N 1 0 1 1         $  s
M1   26000.42c -0.74 24000.42c -0.18 28000.42c -0.08 $ Stainless steel
M2   1001.42c 2.0  8016.42c 1.0   $ Pure water
MT2 lwtr.01
C Tallying energy fluence (nX where X is type (2) and n an ID: {1, 2, ...})
F12:N (1 2)
F22:N (3 4)
C F32:N (1 2 3 4)
C Tally energy bins for all tallies (log 10 space)
E0 1E-9 1E-8 1E-7 1E-6 1E-5 1E-4 1E-3 1E-2 1E-1 1 10
C MC type
MODE N
C Thermal neutron induced fission of 235U at (0,0,22mm)
SDEF POS=0.0 0.0 2.2 ERG=D1
SP1  -3 0.988 2.249
C NUMBER OF PARTICLE HISTORIES TO RUN
NPS  210000
PRDMP 0 0 1 1 0

In [ ]:
clean()

In [ ]:
prettify(mcnp("inp=1.ip", "mctal=1.b.ta"))

# Part 3

In [ ]:
part_3_template =\
"""MESSAGE:

{description}
C Cells
1 4 -19.2 -3 -2 11 $ Uranium source 1
2 4 -19.2 -4 -2 11 $ Uranium source 2
3 4 -19.2 -5 -2 11 $ Uranium source 3
4 4 -19.2 -6 -2 11 $ Uranium source 4
5 3 -{moderator_density} 13 -14 15 -16 11 -12 ((3 4 5 6):2) $ Moderator
6 1 -7.92 7 -8 9 -10 17 -18 (-13:14:-15:16:-11) $ Wall
7 2 -2.3 -17 19 -20 $ Concrete floor
C Void region within bounding volume, outside of the bucket, including air gap:
8 0      (17 -18 -20 (-7:8:-9:10)):(13 -14 15 -16 12 -18) 
9 0      (20:-19:18) $ Bounding cylindrical void region of 0 neutron importance

C Source surfaces
2 PZ 25.2
3 C/Z -20 0 7.5
4 C/Z 0 20 7.5
5 C/Z 20 0 7.5
6 C/Z 0 -20 7.5
C External surfaces
7 PX -50.2
8 PX 50.2
9 PY -50.2
10 PY 50.2
11 PZ 0.2
12 PZ 58.0
C Internal surfaces
13 PX -50
14 PX 50
15 PY -50
16 PY 50
17 PZ 0
18 PZ 60
C Concrete floor lower surface
19 PZ -150.0 
20 CZ 200

MODE N
KCODE 1000 1.0 200 1000
KSRC 20 0 12.7 -20 0 12.7 0 20 12.7 0 -20 12.7
M1  26000.42c -0.74 24000.42c -0.18 28000.42c -0.08 $ Stainless steel
M2  8016.42c -0.53 14000.42c -0.34 20000.42c -0.10 1001.42c -0.03 $ Concrete
M3  {moderator_def}
MT3 {moderator_treatment}
M4  92238.42c -{x_238} 92235.42c -{x_235} $ 80% 238U, 20% 235U
IMP:N 1 1 1 1 1 1 1 1 0"""

# Define async running tools

In [ ]:
import ipywidgets as widgets 
from plumbum import BG
from re import compile
import asyncio

tab = widgets.Tab()

k_pattern = compile(f"final k\(col\/abs\/trk len\) = (\d+\.\d+)     std dev = (\d+\.\d+)") 
err_pattern = compile(f"fatal error.  (.+)\n") 

def parse_stdout_for_k_eff(string):
    match = k_pattern.search(string)
    if match is None:
        return err_pattern.search(string).group()
    return match.group() 


async def wait_proc(proc, sleep_interval=0.2):
    while not proc.poll():
        await asyncio.sleep(sleep_interval)


def run_mcnp_async(input_name, tally_name):
    async def coro():
        try:
            label = widgets.Label('Waiting ...')
            tab.children = [*tab.children, label]
            tab.set_title(len(tab.children)-1, input_name)

            process = mcnp[f"inp={input_name}", f"mctal={tally_name}"] & BG
            await wait_proc(process)

            result = parse_stdout_for_k_eff(process.stdout)
            label.value = result
            return result
        
        finally:
            import traceback
            traceback.print_exc()
    return asyncio.ensure_future(coro())

# 3.a

In [ ]:
x_235 = 0.2
source_info = dict(x_235=x_235, x_238=1-x_235)
water_info = dict(moderator_def="1001.42c 2 8016.42c 1",
                  moderator_treatment="lwtr.01",
                  moderator_density=1.0)

(mcnp_dir/"3.a.ip").write_text(
    part_3_template.format(**water_info, **source_info, description="Water moderated example")
)

# 3.c Replace water with graphite

In [ ]:
graphite_info = dict(moderator_def="6012.42c 1.0   $ Graphite", 
                     moderator_treatment="GRPH.01",
                     moderator_density=1.7)

(mcnp_dir/"3.c.ip").write_text(
    part_3_template.format(**graphite_info, **source_info, description="Graphite moderated example")
)

# 3.d Graphite and Uranium increased enrichment

In [ ]:
x_235 = 0.25
source_info = dict(x_235=x_235, x_238=1-x_235)

(mcnp_dir/"3.d.1.ip").write_text(
    part_3_template.format(**water_info, **source_info, description="Water moderated example")
)

(mcnp_dir/"3.d.2.ip").write_text(
    part_3_template.format(**graphite_info, **source_info, description="Graphite moderated example")
)

# Run simulations

In [ ]:
async def as_finished(futures):
    pending = set(futures)
    
    queue = asyncio.Queue()
    def _on_done(fut):
        queue.put_nowait(fut)
    
    for f in futures:
        f.add_done_callback(_on_done)
        
    while pending:
        fut = await queue.get()
        yield fut
        pending.remove(fut)
        

async def print_as_finished(*tasks):
    import re
    p = re.compile(r"final k\(col/abs/trk len\) = ([^ ]+)     std dev = ([^ ]+)")
    async for t in as_finished(tasks):
        i = tasks.index(t)

        x, dx = p.match(t.result()).groups((1,2))
        print(rf"result[{i}] = \num{{{x} \pm {dx}}}")

In [ ]:
f1 = run_mcnp_async("3.a.ip", "tal3.a")
f2 = run_mcnp_async("3.c.ip", "tal3.c")
f3 = run_mcnp_async("3.d.1.ip", "tal3.d.1")
f4 = run_mcnp_async("3.d.2.ip", "tal3.d.2")
asyncio.ensure_future(print_as_finished(f1, f2, f3, f4))
tab

# Plotting

In [ ]:
from mctal import tokenizer
from derp import parse
import numpy as np
from mctal import g

from bokeh.palettes import Spectral10
from bokeh.io import export_svgs
from bokeh.models import Whisker, ColumnDataSource

colours = iter(Spectral10)


fig = plt.figure(x_axis_type='log', x_axis_label="Energy /MeV", y_axis_label="Flux (1/cm^2)", 
                 plot_width=1200, plot_height=400, 
                 tools='crosshair,pan,wheel_zoom,box_zoom,reset,hover',
                 output_backend='svg')

name = "tallies.svg"
for p in mcnp_dir.glob("1.b.ta"):
    tokens = [*tokenizer.tokenize_file(p)]
    mctal = next(iter(parse(g.mctal, tokens)))

    for i, tally in enumerate(mctal.tallies):
        edges = np.array(tally.energies.values)
        x = (np.r_[0, edges[:-1]] + edges) / 2        
        data = np.array(tally.data)[:-1]
        y,err = data.T
        
#         area = (y * edges).sum()
#         y/=area
        source_error = ColumnDataSource(data=dict(base=x, lower=y*(1-err), upper=y*(1+err)))

        fig.add_layout(
            Whisker(source=source_error, base="base", upper="upper", lower="lower")
        )
        fig.line(x, y, color=next(colours), legend=f'Tally {tally.problem_id}')
        print(f'Tally {tally.problem_id}', np.sqrt(((err*y)**2).sum()))
        
plt.show(fig)
export_svgs(fig, name)
!inkscape {name} -e {name[:-4]}.png --without-gui

In [ ]:
def f(E, a, b):
    return np.exp(-E/a)*np.sinh(np.sqrt(b*E))
def fm(E, Em, Km):
    return Km * np.sqrt(E)*np.exp(-E/Em)
def fw(E,aw,bw,Kw):
    return Kw * np.exp(-E/aw)*np.sinh(np.sqrt(bw*E))
def f(E,aw,bw,Kw,Km,Em,wm):
    return wm*fm(E,Em,Km) + (1-wm)*fw(E,aw,bw,Kw)

x = np.logspace(np.log10(0.1*MeV), np.log10(14*MeV), 1000) 
y = f(x,
      aw=.6859*MeV,
      bw=9.366/MeV,
      Km=1,
      Kw=1,
      Em=1.316 * MeV,
      wm=.7424
     )


fig = plt.figure(x_axis_type='log', x_axis_label="Energy /MeV", y_axis_label="f(x)", 
                 plot_width=1200, plot_height=400, 
                 tools='crosshair,pan,wheel_zoom,box_zoom,reset,hover',
                 output_backend='svg')
fig.line(x, y)
plt.show(fig)
export_svgs(fig, 'watt_spectrum_235_u.svg')
!inkscape 'watt_spectrum_235_u.svg' -e 'watt_spectrum_235_u.png' --without-gui
# !inkscape {name} -e {name[:-4]}.png --without-gui!

In [ ]:
# plot two exponentials against one another

In [ ]:
from contextlib import contextmanager

@contextmanager
def preserve_fs():
    files = remote['ls']().split()
    yield
    new_files = set(remote['ls']().split()).difference(files)
    if new_files:
        remote['rm'](*new_files)

In [ ]:
# with preserve_fs():
#     prettify(mcnp("inp=hc.ip", "mctal=hc.ta"))

## Emin's work

In [ ]:
%%writefile mcnp/hc.ip
Hazel Carter - Problem 1
C
C
C CELL CARDS
C FORMAT: [Cell id number][Material id number][Material density][Surfaces]$[Comment]
C  
1 1 -7.92 6 -8 2 -4 9 -11 (-5:7:-1:3:-10) $Bucket Walls - Stainless Steel [density 7.92g/cm^3]
C
2 0 (-6:8:-2:4:11:-13)  $Void outside Bucket
C
3 2 -1.0 5 -7 1 -3 10 -12  $Water inside Bucket - Pure Water [density 1 g/cm^3]
C
4 0 5 -7 1 -3 12 -11  $Air gap - void
C
5 3 -2.3 -9 13 6 -8 2 -4 $Concrete floor - Concrete [density 2.3 g/cm^3]
C
6 4 -19.2 10 -14 -18 $Uranium Rod 1
7 4 -19.2 10 -15 -18 $Uranium Rod 2
8 4 -19.2 10 -16 -18 $Uranium Rod 3
9 4 -19.2 10 -17 -18 $Uranium Rod 4
C

C
C SURFACE CARDS
C FORMAT: [Surface id number][Surface type][Characteristic dimensions, cm]
C
C Stainless Steel Bucket:
1     PX   -50.0
2     PX   -50.2
3     PX   50.0
4     PX   50.2
5     PY   -50.0
6     PY   -50.2
7     PY   50.0
8     PY   50.2
9     PZ   0.0
10    PZ   0.2
11    PZ   60.0 $Bucket Top
C Water Level:
12    PZ   58.0 $Water Surface
C
C Concrete floor:
13    PZ -60.0
C
C Uranium Rods:
14 C/Z 0 -20 7.5  $Cylinder of radius 7.5 parrel to z axis 
15 C/Z 0 20 7.5
16 C/Z 20 0 7.5
17 C/Z -20 0 7.5
18 PZ 25.2
C

C
C CELL IMPORTANCE MAP  
C FORMAT: [Particle type][Importance of Cell in order they appear in file]
IMP:N 1 0 1 1 1 1 1 1 1
C
C
C MATERIAL CARDS
C FORMAT: M[Material id number][ZZ][AAA][id][Concentration]
C
M1   26000.42c -0.74 24000.42c -0.18 28000.42c -0.08 $Stainless Steel: 74% iron, 18% chromium, 8% nickel.
M2   1001.42c 2.0 8016.42c 1.0 $Pure Water: 2 hyrdogen, 1 oxygen.
M3   8016.42c -0.53 14000.42c -0.34 20000.42c -0.10 1001.42c -0.03 $Concrete: 53% oxygen, 34% silicon, 10% calcium, 3% hydrogen.
M4   92238.42c -0.8 92235.42c -0.2 $Enriched Uranium: 80% U-238, 20% U-235.
C
C
C MODE CARD
C FORMAT: MODE[Particle Type]
MODE N $Neutrons Only                        
C
C
C SOURCE DEFINITION (KCODE)
C FORMAT: [NSRCK][RKK][IKZ][KCT][MSRK][KNRM][MRKP][KC8]
C
C KCODE 1000 0.8 100 1100
C KSRC 20 20 12.5 20 -20 12.5 -20 20 12.5 -20 -20 12.5 

In [ ]:
tab.children = []
tab

In [ ]:
from itertools import count

async def launch():
    counter = count()

    futs = []
    for x_235 in [0.2, 0.25]:
        source_info = dict(x_235=x_235, x_238=1-x_235)

        for mod_info in (water_info, graphite_info):
            i = next(counter)
            f_name = f"em.{i}.ip"

            (mcnp_dir/f_name).write_text(
                emin_template.format(**mod_info, **source_info, description="Water moderated example")
            )
            fut = run_mcnp_async(f_name, "emin.tal")
            futs.append(fut)
            await asyncio.sleep(3.0)
    await print_as_finished(futs)
            
asyncio.ensure_future(launch())

In [ ]:
import pexpect
emin_mcnp_dir = pathlib.Path.cwd() / "emin"

user = "exh478"
password = "1Forrest1"
p = pexpect.spawn('bash')
p.sendline(f'sshfs {user}@{domain}:mcnpdir {emin_mcnp_dir} -p {port}')
p.expect("password:")
p.sendline(password)
p.sendline('exit')
p.expect_exact(pexpect.EOF)

for i in range(1, 5):
    fname = f"em.{i}.ip"
    (emin_mcnp_dir/fname).write_text((mcnp_dir / fname).read_text())

In [ ]:
from pygments.lexer import RegexLexer
from pygments.token import *

class DiffLexer(RegexLexer):
    name = 'Diff'
    aliases = ['diff']
    filenames = ['*.diff']

    tokens = {
        'root': [
            (r' .*\n', Text),
            (r'\+.*\n', Generic.Inserted),
            (r'-.*\n', Generic.Deleted),
            (r'@.*\n', Generic.Subheading),
            (r'Index.*\n', Generic.Heading),
            (r'=.*\n', Generic.Heading),
            (r'.*\n', Text),
        ]
    }
